---

In [ ]:
import torch
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append("/home/twhit/smc_object_detection/")

from smc.metrics import match_catalogs, compute_precision_recall_f1
from bliss.catalog import convert_nmgy_to_mag, convert_mag_to_nmgy

---

### **Load in images, true catalogs, and SMC results**

We have 70,000 SMC catalogs for each image, but we'll just use 10,000.

In [ ]:
num_batches = len(glob.glob("smc/counts_*.pt"))
batch_size = 10
num_images = num_batches * batch_size

prior_max_objects = 6
num_catalogs_per_count = 10000
num_catalogs = num_catalogs_per_count * (prior_max_objects + 1)

images = torch.load('../data/tiles.pt')[:num_images]
pruned_counts = torch.load('../data/counts_magcut.pt')[:num_images].int()
pruned_locs = torch.load('../data/locs_magcut.pt')[:num_images]
pruned_fluxes = torch.load('../data/fluxes_magcut.pt')[:num_images]

smc_unpruned_counts = torch.zeros([num_batches, batch_size, num_catalogs])
smc_unpruned_locs = torch.zeros([num_batches, batch_size, num_catalogs, prior_max_objects, 2])
smc_unpruned_fluxes = torch.zeros([num_batches, batch_size, num_catalogs, prior_max_objects])
smc_posterior_predictive_total_flux = torch.zeros([num_batches, batch_size, num_catalogs])
smc_runtime = torch.zeros([num_batches, batch_size])
smc_num_iters = torch.zeros([num_batches, batch_size])

for b in range(num_batches):
    smc_unpruned_counts[b] = torch.load(f'smc/counts_{b}.pt')
    smc_unpruned_locs[b] = torch.load(f'smc/locs_{b}.pt')
    smc_unpruned_fluxes[b] = torch.load(f'smc/fluxes_{b}.pt')
    smc_posterior_predictive_total_flux[b] = torch.load(f'smc/posterior_predictive_total_flux_{b}.pt')
    smc_runtime[b] = torch.load(f'smc/runtime_{b}.pt')
    smc_num_iters[b] = torch.load(f'smc/num_iters_{b}.pt')

torch.manual_seed(1)
num_catalogs_for_inference = 10000
indexes = torch.randint(0, smc_unpruned_counts.shape[-1], size = [num_catalogs_for_inference])

smc_unpruned_counts = smc_unpruned_counts.flatten(0,1)[:,indexes].int()
smc_unpruned_locs = smc_unpruned_locs.flatten(0,1)[:,indexes,:,:]
smc_unpruned_fluxes = smc_unpruned_fluxes.flatten(0,1)[:,indexes,:]
smc_posterior_predictive_total_flux = smc_posterior_predictive_total_flux.flatten(0,1)
smc_runtime = smc_runtime.flatten(0,1)
smc_num_iters = smc_num_iters.flatten(0,1)

In [ ]:
tile_dim = 8
pad = 1

smc_mask = torch.logical_and(smc_unpruned_locs > 0, smc_unpruned_locs < tile_dim).all(-1)
smc_pruned_counts = smc_mask.sum(-1).int()

smc_pruned_locs = smc_mask.unsqueeze(-1) * smc_unpruned_locs
smc_locs_mask = (smc_pruned_locs != 0).int()
smc_locs_index = torch.sort(smc_locs_mask, dim=-2, descending=True)[1]
smc_pruned_locs = torch.gather(smc_pruned_locs, dim=-2, index=smc_locs_index)

smc_pruned_fluxes = smc_mask * smc_unpruned_fluxes
smc_fluxes_mask = (smc_pruned_fluxes != 0).int()
smc_fluxes_index = torch.sort(smc_fluxes_mask, dim=-1, descending=True)[1]
smc_pruned_fluxes = torch.gather(smc_pruned_fluxes, dim=-1, index=smc_fluxes_index)

---

### **Load in SEP results**

In [ ]:
sep_unpruned_counts = torch.load('sep/counts.pt')[:num_images].int()
sep_unpruned_locs = torch.load('sep/locs.pt')[:num_images]
sep_unpruned_fluxes = torch.load('sep/fluxes.pt')[:num_images]
sep_runtime = torch.load('sep/runtime.pt')[:num_images]

In [ ]:
sep_mask = torch.logical_and(sep_unpruned_locs > 0, sep_unpruned_locs < tile_dim).all(-1)
sep_pruned_counts = sep_mask.sum(-1)

sep_pruned_locs = sep_mask.unsqueeze(-1) * sep_unpruned_locs
sep_locs_mask = (sep_pruned_locs != 0).int()
sep_locs_index = torch.sort(sep_locs_mask, dim=-2, descending=True)[1]
sep_pruned_locs = torch.gather(sep_pruned_locs, dim=-2, index=sep_locs_index)

sep_pruned_fluxes = sep_mask * sep_unpruned_fluxes
sep_fluxes_mask = (sep_pruned_fluxes != 0).int()
sep_fluxes_index = torch.sort(sep_fluxes_mask, dim=-1, descending=True)[1]
sep_pruned_fluxes = torch.gather(sep_pruned_fluxes, dim=-1, index=sep_fluxes_index)

---

### **Plot attributes**

In [ ]:
smc_color = "steelblue"
smc_marker = 'o'
sep_color = "mediumvioletred"
sep_marker = 'D'

bigfont = 20
mediumfont = 16
smallfont = 14

cmap = 'gray'

linecolor = 'sienna'
linecolor_alt = 'forestgreen'
linewidth = 4
linestyle = 'dashed'

star_marker = '*'
star_cmap = 'Wistia'
star_edge = 'black'
star_size = 400
smc_star_size = 0.25 * star_size
smc_star_alpha = 0.25

---

### **True vs. inferred magnitude distributions**

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (12, 4))

_ = ax[0].hist(convert_nmgy_to_mag(pruned_fluxes[pruned_fluxes>0]),
               bins = 30, color = 'dimgray', density = True)
_ = ax[1].hist(convert_nmgy_to_mag(smc_pruned_fluxes[smc_pruned_fluxes>0][torch.randint(0, smc_pruned_fluxes[smc_pruned_fluxes>0].shape[0], size = [10000])]),
               bins = 30, color = smc_color, density = True)
_ = ax[2].hist(convert_nmgy_to_mag(sep_pruned_fluxes[sep_pruned_fluxes>0]),
               bins = 30, color = sep_color, density = True)

for a in ax.flat:
    _ = a.set_xlabel('r-band magnitude')
    _ = a.set_ylabel('Density')
    _ = a.set_xlim(14, 25)
    _ = a.set_ylim(0, 0.3)

_ = fig.tight_layout()

---

### **Example detections**

We write a function to plot the true catalog, one or multiple SMC catalogs, and the SEP catalog:

In [ ]:
def plot_detections(image_index, arcsinh = False,
                    smc_cat_idx = "all", smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha, smc_star_edge = False,
                    star_minflux = None, star_maxflux = None,
                    ylabel = "", title = True, ylabel_fontsize = bigfont, figsize = (18, 6)):
    fig, ax = plt.subplots(1, 3, figsize = figsize, constrained_layout = True)

    if arcsinh:
        img = images[image_index].arcsinh()
    else:
        img = images[image_index]
    
    if smc_cat_idx == "all":
        smc_cat_idx = torch.arange(0, smc_unpruned_counts.shape[0])
    smc_locs_h = smc_unpruned_locs[image_index, smc_cat_idx, :, 1]
    smc_locs_w = smc_unpruned_locs[image_index, smc_cat_idx, :, 0]
    smc_fluxes = smc_unpruned_fluxes[image_index,smc_cat_idx]
    
    if star_minflux is None:
        star_minflux = min(pruned_fluxes[image_index][pruned_fluxes[image_index]>0].min(), smc_fluxes[smc_fluxes>0].min())
    if star_maxflux is None:
        star_maxflux = max(pruned_fluxes[image_index][pruned_fluxes[image_index]>0].max(), smc_fluxes[smc_fluxes>0].max())
    
    # true catalog
    _ = ax[0].imshow(img, cmap = cmap)
    if title:
        _ = ax[0].set_title("True catalog", fontsize = ylabel_fontsize)
    else:
        _ = ax[0].set_title("", fontsize = ylabel_fontsize)

    _ = ax[0].set_ylabel(ylabel, fontsize = ylabel_fontsize)
        
    flux_cbar = ax[0].scatter(pruned_locs[image_index, :pruned_counts[image_index], 1] - 0.5,
                            pruned_locs[image_index, :pruned_counts[image_index], 0] - 0.5,
                            c = pruned_fluxes[image_index, :pruned_counts[image_index]],
                            vmin = star_minflux, vmax = star_maxflux,
                            cmap = star_cmap, edgecolors = star_edge, marker = star_marker, s = star_size)
    _ = ax[0].spines[['top', 'right', 'bottom', 'left']].set_visible(False)
    _ = ax[0].set_xticks([])
    _ = ax[0].set_yticks([])
    _ = ax[0].hlines(y = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    xmin = 0 - pad - 0.5, xmax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)
    _ = ax[0].vlines(x = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    ymin = 0 - pad - 0.5, ymax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)

    # SMC catalogs
    _ = ax[1].imshow(img, cmap = cmap)
    if title:
        _ = ax[1].set_title("Count-stratified SMC", fontsize = ylabel_fontsize)
    else:
        _ = ax[1].set_title("", fontsize = ylabel_fontsize)
    
    _ = ax[1].scatter(smc_locs_h[smc_locs_h.nonzero(as_tuple=True)] - 0.5,
                    smc_locs_w[smc_locs_w.nonzero(as_tuple=True)] - 0.5,
                    c = smc_fluxes[smc_fluxes.nonzero(as_tuple=True)], cmap = star_cmap,
                    vmin = star_minflux, vmax = star_maxflux,
                    marker = star_marker, s = smc_star_size, alpha = smc_star_alpha, edgecolors = star_edge if smc_star_edge else None)
    _ = ax[1].spines[['top', 'right', 'bottom', 'left']].set_visible(False)
    _ = ax[1].set_xticks([])
    _ = ax[1].set_yticks([])
    _ = ax[1].hlines(y = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    xmin = 0 - pad - 0.5, xmax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)
    _ = ax[1].vlines(x = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    ymin = 0 - pad - 0.5, ymax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)

    # SEP catalog
    _ = ax[2].imshow(img, cmap = cmap)
    if title:
        _ = ax[2].set_title("Source Extractor", fontsize = ylabel_fontsize)
    else:
        _ = ax[2].set_title("", fontsize = ylabel_fontsize)

    _ = ax[2].scatter(sep_unpruned_locs[image_index, :sep_unpruned_counts[image_index], 1] - 0.5,
                      sep_unpruned_locs[image_index, :sep_unpruned_counts[image_index], 0] - 0.5,
                      c = sep_unpruned_fluxes[image_index, :sep_unpruned_counts[image_index]],
                      vmin = star_minflux, vmax = star_maxflux,
                      cmap = star_cmap, edgecolors = star_edge, marker = star_marker, s = star_size)
    _ = ax[2].spines[['top', 'right', 'bottom', 'left']].set_visible(False)
    _ = ax[2].set_xticks([])
    _ = ax[2].set_yticks([])
    _ = ax[2].hlines(y = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    xmin = 0 - pad - 0.5, xmax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)
    _ = ax[2].vlines(x = [0 - pad - 0.5, tile_dim + pad - 0.5],
                    ymin = 0 - pad - 0.5, ymax = tile_dim + pad - 0.5,
                    color = linecolor, linestyles = linestyle, linewidth = linewidth, alpha = 0)

    cbar = fig.colorbar(flux_cbar, ax=ax, orientation='vertical', fraction = 0.014, pad = 0.02)
    cbar.ax.tick_params(labelsize = 0.75 * ylabel_fontsize)
    
    return fig

We compare the true catalog, our ensemble of SMC catalogs, and the SEP catalog for an image with one star, an image with two stars, with three stars, four stars, and five:

In [ ]:
idx = 95

fig = plot_detections(image_index = idx, smc_cat_idx = "all",
                      smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha)

fig.savefig('figures/m71_detections_count1_all.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
idx = 42

fig = plot_detections(image_index = idx, smc_cat_idx = "all",
                      smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha)

fig.savefig('figures/m71_detections_count2_all.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
idx = 152

fig = plot_detections(image_index = idx, smc_cat_idx = "all",
                      smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha)

fig.savefig('figures/m71_detections_count3_all.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
idx = 184

fig = plot_detections(image_index = idx, smc_cat_idx = "all",
                      smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha)

fig.savefig('figures/m71_detections_count4_all.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
idx = 14

fig = plot_detections(image_index = idx, smc_cat_idx = "all",
                      smc_star_size = smc_star_size, smc_star_alpha = smc_star_alpha)

fig.savefig('figures/m71_detections_count5_all.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

---

### **Source count heat maps**

In [ ]:
torch.manual_seed(0)

confmat_magcut = torch.tensor(24.0)
confmat_fluxcut = convert_mag_to_nmgy(confmat_magcut)

confmat_smc_sampled_counts = ((smc_pruned_fluxes != 0) & (smc_pruned_fluxes >= confmat_fluxcut)).sum(-1)
confmat_true_counts = ((pruned_fluxes != 0) & (pruned_fluxes >= confmat_fluxcut)).sum(-1).unsqueeze(-1).repeat(1, smc_pruned_counts.shape[-1])

smc_samples_confmat = torch.histogramdd(
    torch.stack((confmat_smc_sampled_counts, confmat_true_counts), dim=-1).float(),
    bins = (torch.arange(0, smc_pruned_counts.max() + 2).float(), torch.arange(0, pruned_counts.max() + 2).unique().float())
).hist

fig, ax = plt.subplots(1, 1, figsize = (7, 6))

panel = sns.heatmap(smc_samples_confmat.int().numpy(),
                    ax = ax, annot = True, annot_kws={"size": smallfont}, fmt = "d",
                    cmap = sns.light_palette(smc_color, as_cmap=True), cbar = False)
panel.invert_yaxis()

def format_func(value, tick_position):
    if value >= 100000:
        return f"{round(value / 1000, 1):.0f}k"
    if value >= 50:
        return f"{round(value / 1000, 1):.2g}k"
    return str("0")

for text in panel.texts:
    value = int(text.get_text())
    text.set_text(format_func(value, None))

panel.set_xlabel("True source count", fontsize = bigfont)
panel.set_ylabel("Estimated source count", fontsize = bigfont)
panel.tick_params(axis = 'both', length = 0, labelsize = mediumfont)

_ = ax.set_title("Count-stratified SMC posterior samples", fontsize = bigfont)

fig.tight_layout()

fig.savefig('figures/m71_confmat_smc.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
confmat_smc_mode_counts = ((smc_pruned_fluxes != 0) & (smc_pruned_fluxes >= confmat_fluxcut)).sum(-1).mode(-1).values
confmat_true_counts = ((pruned_fluxes != 0) & (pruned_fluxes >= confmat_fluxcut)).sum(-1)

smc_samples_confmat = torch.histogramdd(
    torch.stack((confmat_smc_mode_counts, confmat_true_counts), dim=-1).float(),
    bins = (torch.arange(0, smc_pruned_counts.max() + 2).float(), torch.arange(0, pruned_counts.max() + 2).unique().float())
).hist

fig, ax = plt.subplots(1, 1, figsize = (7, 6))

panel = sns.heatmap(smc_samples_confmat.int().numpy(),
                    ax = ax, annot = True, annot_kws={"size": smallfont}, fmt = "d",
                    cmap = sns.light_palette(smc_color, as_cmap=True), cbar = False)
panel.invert_yaxis()

panel.set_xlabel("True source count", fontsize = bigfont)
panel.set_ylabel("Estimated source count", fontsize = bigfont)
panel.tick_params(axis = 'both', length = 0, labelsize = mediumfont)

_ = ax.set_title("Count-stratified SMC posterior mode", fontsize = bigfont)

fig.tight_layout()

fig.savefig('figures/m71_confmat_smc.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
confmat_sep_counts = ((sep_pruned_fluxes != 0) & (sep_pruned_fluxes >= confmat_fluxcut)).sum(-1)

sep_confmat = torch.histogramdd(
    torch.stack((confmat_sep_counts, confmat_true_counts), dim=-1).float(),
    bins = (torch.arange(0, smc_pruned_counts.max() + 2).float(), torch.arange(0, pruned_counts.max() + 2).unique().float())
).hist

fig, ax = plt.subplots(1, 1, figsize = (7, 6))
panel = sns.heatmap(sep_confmat.int().numpy(),
                    ax = ax, annot = True, annot_kws={"size": smallfont}, fmt = "d",
                    cmap = sns.light_palette(sep_color, as_cmap=True), cbar = False)
panel.invert_yaxis()

panel.set_xlabel("True source count", fontsize = bigfont)
panel.set_ylabel("Estimated source count", fontsize = bigfont)
panel.tick_params(axis = 'both', length = 0, labelsize = mediumfont)

_ = ax.set_title("Source Extractor", fontsize = bigfont)

fig.tight_layout()

fig.savefig('figures/m71synth_confmat_sep.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

---

### **Calibration of total intrinsic flux**

We compute and plot the empirical coverage rate of x% credible intervals formed using CS-SMC samples for $x \in [0.05,...,0.95]$

In [ ]:
intervals = [c*0.05 for c in range(1,20)]
coverage_rate = []

smc_masked_tif = smc_pruned_fluxes[pruned_counts>0].sum(-1)
masked_tif = pruned_fluxes[pruned_counts>0].sum(-1)

for i in intervals:
    qlow = smc_masked_tif.quantile(0.5 - i / 2, dim = -1)
    qhigh = smc_masked_tif.quantile(0.5 + i / 2, dim = -1)
    covered = torch.logical_and(qlow <= masked_tif, qhigh >= masked_tif)
    
    coverage_rate.append(covered.float().mean())

[(f"nominal coverage probability = {round(i,2)}, empirical coverage probability = {round(r.item(),2)}") for (i,r) in zip(intervals, coverage_rate)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 6))

_ = ax.axline((0,0), slope = 1, linestyle = ':', color = 'black', zorder = 0)
_ = ax.scatter(intervals, coverage_rate, label = 'Count-stratified SMC',
               color = smc_color, marker = smc_marker, s = 80, zorder = 1)

_ = ax.set_xlim((0,1))
_ = ax.set_ylim((0,1))

_ = ax.set_xlabel('Nominal coverage probability', fontsize = bigfont)
_ = ax.set_ylabel('Empirical coverage probability', fontsize = bigfont)

_ = ax.tick_params(axis = 'both', labelsize = mediumfont)

_ = ax.legend(loc = 'upper left', prop = {'size': smallfont}, handletextpad = 0.1)

_ = ax.spines[['top', 'right']].set_visible(False)

fig.tight_layout()

fig.savefig('figures/m71_totalflux_coverage_smc.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

And to demonstrate this achievement of nominal coverage, we plot the 90% credible intervals for SMC and compare them to Source Extractor's point estimates:

In [ ]:
interval = 0.9

# mask intervals that contain zero for aesthetic purposes
mask = smc_pruned_fluxes.sum(-1).quantile(0.05, dim=-1) > 0

q05 = smc_pruned_fluxes[mask].sum(-1).quantile(0.5 - interval / 2, dim = -1)
q95 = smc_pruned_fluxes[mask].sum(-1).quantile(0.5 + interval / 2, dim = -1)
covered = torch.logical_and(q05 <= pruned_fluxes[mask].sum(-1),
                            q95 >= pruned_fluxes[mask].sum(-1))

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

true_total_intrinsic_flux = pruned_fluxes[mask].sum(-1)

covered_legend = False
uncovered_legend = False
# to avoid one color from dominating, plot by index instead of in two groups
for i in range(pruned_fluxes[mask].shape[0]):
    color = smc_color if covered[i] else 'dimgray'
    if (covered[i]) and (not covered_legend):
        covered_legend = True
        _ = ax.vlines(x = true_total_intrinsic_flux[i],
                      ymin = q05[i], ymax = q95[i], color = color, alpha = 1, linewidth = 1,
                      label = f"covers ({round(100 * coverage_rate[-2].item(), 1)}%)")
    elif (not covered[i]) and (not uncovered_legend):
        uncovered_legend = True
        _ = ax.vlines(x = true_total_intrinsic_flux[i],
                      ymin = q05[i], ymax = q95[i], color = color, alpha = 1, linewidth = 1,
                      label = f"does not cover ({round(100 * (1 - coverage_rate[-2]).item(), 1)}%)")
    else:
        _ = ax.vlines(x = true_total_intrinsic_flux[i],
                    ymin = q05[i], ymax = q95[i], color = color, alpha = 1, linewidth = 1)

_ = ax.axline((0,0), slope = 1, color = 'black', linestyle = ':', linewidth = 1)

_ = ax.set_xscale('log')
_ = ax.set_yscale('log')
_ = ax.set_xlabel('True total intrinsic flux', fontsize = bigfont)
_ = ax.set_ylabel('Estimated total intrinsic flux', fontsize = bigfont)
_ = ax.tick_params(axis = "both", labelsize = mediumfont)

_ = ax.spines[['top', 'right']].set_visible(False)

leg = ax.legend(loc = 'upper left', prop = {'size': smallfont}, title = "Count-stratified SMC interval...", title_fontsize = smallfont)
leg._legend_box.align = 'left'

fig.tight_layout()

fig.savefig('figures/m71_totalflux_intervals_smc.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# to avoid one color from dominating, plot by index instead of in two groups
_ = ax.scatter(x = pruned_fluxes[mask].sum(-1), y = sep_pruned_fluxes[mask].sum(-1),
               color = sep_color, marker = sep_marker, s = 10, alpha = 0.9, label = "Source Extractor")

_ = ax.axline((0,0), slope = 1, color = 'black', linestyle = ':', linewidth = 1)

_ = ax.set_xscale('log')
_ = ax.set_yscale('log')
_ = ax.set_xlabel('True total intrinsic flux', fontsize = bigfont)
_ = ax.set_ylabel('Estimated total intrinsic flux', fontsize = bigfont)

_ = ax.tick_params(axis = "both", labelsize = mediumfont)
_ = ax.legend(loc = 'upper left', prop = {'size': smallfont},
              handletextpad = 0.1, markerscale = 2)

_ = ax.spines[['top', 'right']].set_visible(False)

fig.tight_layout()

fig.savefig('figures/m71_totalflux_sep.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
# simulation-based calibration plot
sbc_mh_totalfluxes = (masked_tif.unsqueeze(-1) <= smc_masked_tif).float().mean(-1)

fig, ax = plt.subplots(1, 1, figsize = (6, 6))
_ = ax.hist(sbc_mh_totalfluxes, bins = 10, density = True)

In [ ]:
# true total observed flux vs mean posterior predictive total observed flux
fig, ax = plt.subplots(1, 1, figsize = (6, 6))

posterior_predictive_mask = images.sum([-1,-2]) < images.sum([-1,-2]).quantile(0.99)

_ = ax.scatter(x = images[posterior_predictive_mask].sum([-1,-2]),
               y = smc_posterior_predictive_total_flux[posterior_predictive_mask].mean(-1),
               color = smc_color, s = 5, alpha = 1, label = "CS-SMC")

_ = ax.axline((0,0), slope = 1, color = 'black', linestyle = ':', linewidth = 1)

_ = ax.set_xscale('log')
_ = ax.set_yscale('log')

_ = ax.set_xlabel('True total observed flux', fontsize = bigfont)
_ = ax.set_ylabel('Mean posterior predictive\ntotal observed flux', fontsize = bigfont)
_ = ax.tick_params(axis = 'both', labelsize = mediumfont)

---

### **Precision, recall, and F1**

We match sources that are within half a pixel of a true source and have an r-band mag within half a unit. We use mag bins ranging from 17 to 24.

In [ ]:
mag_bins = torch.arange(18, 24.5, 0.5)
mags_tol = 0.5
locs_tol = 0.5

In [ ]:
torch.manual_seed(0)

smc_matching_results = match_catalogs(pruned_counts, pruned_locs, pruned_fluxes,
                                      smc_pruned_counts, smc_pruned_locs, smc_pruned_fluxes,
                                      250, locs_tol, mags_tol, mag_bins)
smc_true_total, smc_true_matches, smc_est_total, smc_est_matches = smc_matching_results

smc_precision, smc_recall, smc_f1 = compute_precision_recall_f1(smc_true_total, smc_true_matches, smc_est_total, smc_est_matches)

In [ ]:
num_boot_samples = 10000
boot_mean_precision = torch.zeros(num_boot_samples, len(mag_bins))
boot_mean_recall = torch.zeros(num_boot_samples, len(mag_bins))
boot_mean_f1 = torch.zeros(num_boot_samples, len(mag_bins))

for b in range(num_boot_samples):
    boot_idx = torch.randint(0, smc_precision.shape[0], size = [smc_precision.shape[0]])
    boot_mean_precision[b] = smc_precision[boot_idx].mean(0)
    boot_mean_recall[b] = smc_recall[boot_idx].mean(0)
    boot_mean_f1[b] = smc_f1[boot_idx].mean(0)

alpha = 0.1 # 90% confidence intervals
num_multiple_comparisons = 3 * len(mag_bins) # Bonferroni correction

smc_precision_lower = torch.quantile(boot_mean_precision, alpha / num_multiple_comparisons, dim = 0)
smc_precision_mean = boot_mean_precision.mean(dim = 0)
smc_precision_upper = torch.quantile(boot_mean_precision, 1 - alpha / num_multiple_comparisons, dim = 0)

smc_recall_lower = torch.quantile(boot_mean_recall, alpha / num_multiple_comparisons, dim = 0)
smc_recall_mean = boot_mean_recall.mean(dim = 0)
smc_recall_upper = torch.quantile(boot_mean_recall, 1 - alpha / num_multiple_comparisons, dim = 0)
smc_f1_lower = torch.quantile(boot_mean_f1, alpha / num_multiple_comparisons, dim = 0)
smc_f1_mean = boot_mean_f1.mean(dim = 0)
smc_f1_upper = torch.quantile(boot_mean_f1, 1 - alpha / num_multiple_comparisons, dim = 0)

In [ ]:
sep_matching_results = match_catalogs(pruned_counts, pruned_locs, pruned_fluxes,
                                      sep_pruned_counts.unsqueeze(1), sep_pruned_locs.unsqueeze(1), sep_pruned_fluxes.unsqueeze(1),
                                      1, locs_tol, mags_tol, mag_bins)
sep_true_total, sep_true_matches, sep_est_total, sep_est_matches = sep_matching_results

sep_precision, sep_recall, sep_f1 = compute_precision_recall_f1(sep_true_total, sep_true_matches, sep_est_total, sep_est_matches)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (18, 6*1.075))

ms = 8

_ = ax[0].plot(mag_bins, smc_precision_mean, color = smc_color, marker = smc_marker, ms = ms, label = "Count-stratified SMC")
_ = ax[0].plot(mag_bins, sep_precision.squeeze(), color = sep_color, marker = sep_marker, ms = 0.9*ms, label = "Source Extractor")
_ = ax[0].fill_between(mag_bins, smc_precision_lower, smc_precision_upper, color = smc_color, alpha = 0.5)
_ = ax[0].set_ylabel('Precision', fontsize = bigfont)

_ = ax[1].plot(mag_bins, smc_recall.mean(0), color = smc_color, marker = smc_marker, ms = ms, label = "Count-stratified SMC")
_ = ax[1].plot(mag_bins, sep_recall.squeeze(), color = sep_color, marker = sep_marker, ms = 0.9*ms, label = "Source Extractor")
_ = ax[1].fill_between(mag_bins, smc_recall_lower, smc_recall_upper, color = smc_color, alpha = 0.5)
_ = ax[1].set_ylabel('Recall', fontsize = bigfont)

_ = ax[2].plot(mag_bins, smc_f1_mean, color = smc_color, marker = smc_marker, ms = ms, label = "Count-stratified SMC")
_ = ax[2].plot(mag_bins, sep_f1.squeeze(), color = sep_color, marker = sep_marker, ms = 0.9*ms, label = "Source Extractor")
_ = ax[2].fill_between(mag_bins, smc_f1_lower, smc_f1_upper, color = smc_color, alpha = 0.5)
_ = ax[2].set_ylabel('F1 score', fontsize = bigfont)

for a in ax.flat:
    _ = a.tick_params(axis = "both", labelsize = 0.8*smallfont)
    _ = a.set_xticks(ticks = mag_bins,
                     labels = [f'<{mag_bins[0]}'] + [f'[{mag_bins[t].item()}, {mag_bins[t+1]})' for t in range(len(mag_bins)-1)],
                     rotation = 45, ha = 'right', rotation_mode = "anchor")
    _ = a.set_xlabel('r-band magnitude\n(dimmer $\Longrightarrow$)', fontsize = bigfont)
    _ = a.spines[['top', 'right']].set_visible(False)
    _ = a.set_ylim((0,1.05))

handles, labels = ax[0].get_legend_handles_labels()
_ = fig.legend(handles, labels, loc = 'upper center', ncol = 2, bbox_to_anchor = (0.5, 1.075), fontsize = smallfont)

_ = fig.tight_layout()

fig.savefig('figures/m71_matching.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

---

### **Cumulative luminosity functions**

In [ ]:
torch.manual_seed(0)

clf_mag_bins = torch.arange(14.5, 24.5, 0.5)

clf_smc_matches = match_catalogs(pruned_counts, pruned_locs, pruned_fluxes,
                                 smc_pruned_counts, smc_pruned_locs, smc_pruned_fluxes,
                                 50, locs_tol, mags_tol, clf_mag_bins)
clf_smc_true_total, clf_smc_true_matches, clf_smc_est_total, clf_smc_est_matches = clf_smc_matches

clf_sep_matches = match_catalogs(pruned_counts, pruned_locs, pruned_fluxes,
                                 sep_pruned_counts.unsqueeze(1), sep_pruned_locs.unsqueeze(1), sep_pruned_fluxes.unsqueeze(1),
                                 1, locs_tol, mags_tol, clf_mag_bins)
clf_sep_true_total, clf_sep_true_matches, clf_sep_est_total, clf_sep_est_matches = clf_sep_matches

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (14, 6 * 1.075))

_ = ax[0].plot(clf_mag_bins, clf_smc_true_total.sum(0)[0].cumsum(0), label = "True catalog",
               color = 'black', linewidth = 1.5, zorder = 100)
_ = ax[0].plot(clf_mag_bins, clf_smc_est_total.sum(0)[0].cumsum(-1), label = "Count-stratified SMC",
               color = smc_color, linestyle = 'dashed', linewidth = 1)
_ = ax[0].plot(clf_mag_bins, clf_smc_est_total.sum(0)[1:].cumsum(-1).permute(1, 0),
               color = smc_color, linestyle = 'dashed', linewidth = 1)
_ = ax[0].plot(clf_mag_bins, clf_sep_est_total.sum(0).cumsum(-1).permute(1, 0), label = "Source Extractor",
               color = sep_color, linestyle = 'dashed', linewidth = 1)
_ = ax[0].set_title('All sources', fontsize = mediumfont)
_ = ax[0].set_ylim(0,1200)
_ = ax[1].plot(clf_mag_bins, clf_smc_true_matches.sum(0)[0].cumsum(0),
               color = 'black', linewidth = 1.5, zorder = 100)
_ = ax[1].plot(clf_mag_bins, clf_smc_est_matches.sum(0)[0].cumsum(-1),
               color = smc_color, linestyle = 'dashed', linewidth = 1)
_ = ax[1].plot(clf_mag_bins, clf_smc_est_matches.sum(0)[1:].cumsum(-1).permute(1, 0),
               color = smc_color, linestyle = 'dashed', linewidth = 1)
_ = ax[1].plot(clf_mag_bins, clf_sep_est_matches.sum(0).cumsum(-1).permute(1, 0),
               color = sep_color, linestyle = 'dashed', linewidth = 1)
_ = ax[1].set_title('Matched sources', fontsize = mediumfont)

for a in ax.flat:
    _ = a.tick_params(axis = "both", labelsize = 0.8*smallfont)
    _ = a.set_xticks(ticks = clf_mag_bins,
                     labels = [f'<{clf_mag_bins[t].item()}' for t in range(len(clf_mag_bins))],
                     rotation = 45, ha = 'right', rotation_mode = "anchor")
    _ = a.set_xlabel('r-band magnitude\n(dimmer $\Longrightarrow$)', fontsize = bigfont)
    _ = a.set_ylabel('Cumulative number of sources', fontsize = bigfont)
    _ = a.spines[['top', 'right']].set_visible(False)

handles, labels = ax[0].get_legend_handles_labels()
_ = fig.legend(handles, labels, loc = 'upper center', ncol = 3, bbox_to_anchor = (0.5, 1.075), fontsize = smallfont)

fig.savefig('figures/m71_clf.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

---

### **Runtime and number of SMC iterations**

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (18, 6))

_ = ax[0].scatter(pruned_counts, smc_runtime, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[0].set_ylim((0, 120))
_ = ax[0].set_xlabel("True source count", fontsize = bigfont)
_ = ax[0].set_ylabel("Runtime (seconds)", fontsize = bigfont)
_ = ax[0].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[0].spines[['top', 'right']].set_visible(False)

_ = ax[1].scatter(pruned_fluxes.sum(-1), smc_runtime, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[1].set_ylim((0, 120))
_ = ax[1].set_xscale('log')
_ = ax[1].set_xlabel("True total intrinsic flux", fontsize = bigfont)
_ = ax[1].set_ylabel("Runtime (seconds)", fontsize = bigfont)
_ = ax[1].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[1].spines[['top', 'right']].set_visible(False)

_ = ax[2].scatter(images.flatten(1,2).sum(-1), smc_runtime, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[2].set_ylim((0, 120))
_ = ax[2].set_xscale('log')
_ = ax[2].set_xlabel("Sum of image pixel intensities", fontsize = bigfont)
_ = ax[2].set_ylabel("Runtime (seconds)", fontsize = bigfont)
_ = ax[2].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[2].spines[['top', 'right']].set_visible(False)

fig.tight_layout()

fig.savefig('figures/m71_runtime.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (18, 6))

_ = ax[0].scatter(pruned_counts, smc_num_iters, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[0].set_ylim((0, 60))
_ = ax[0].set_xlabel("True source count", fontsize = bigfont)
_ = ax[0].set_ylabel("Number of SMC iterations", fontsize = bigfont)
_ = ax[0].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[0].spines[['top', 'right']].set_visible(False)

_ = ax[1].scatter(pruned_fluxes.sum(-1), smc_num_iters, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[1].set_ylim((0, 60))
_ = ax[1].set_xscale('log')
_ = ax[1].set_xlabel("True total intrinsic flux", fontsize = bigfont)
_ = ax[1].set_ylabel("Number of SMC iterations", fontsize = bigfont)
_ = ax[1].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[1].spines[['top', 'right']].set_visible(False)

_ = ax[2].scatter(images.flatten(1,2).sum(-1), smc_num_iters, color = smc_color, marker = smc_marker, alpha = 0.5)
_ = ax[2].set_ylim((0, 60))
_ = ax[2].set_xscale('log')
_ = ax[2].set_xlabel("Sum of image pixel intensities", fontsize = bigfont)
_ = ax[2].set_ylabel("Number of SMC iterations", fontsize = bigfont)
_ = ax[2].tick_params(axis = "both", labelsize = mediumfont)
_ = ax[2].spines[['top', 'right']].set_visible(False)

fig.tight_layout()

fig.savefig('figures/m71_iterations.png', dpi = 250, transparent = True, bbox_inches = 'tight', pad_inches = 0)

---